# MUST RUN AT THE START OF EVERYTHING

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
database_str = "sqlite:///" + os.environ['WORKINGPATH'] + "/Database/epilepsy.db"
os.environ['SNORKELDB'] = database_str

from snorkel import SnorkelSession
session = SnorkelSession()

# Parse the Pubmed Abstracts

The code below is designed to read and parse data gathered from pubtator. Pubtator outputs their annotated text in xml format, so that is the standard file format we are going to use. 

In [2]:
from epilepsy_utils import XMLMultiDocPreprocessor
import os
working_path = os.environ['WORKINGPATH']
xml_parser = XMLMultiDocPreprocessor(
    path= working_path + '/Database/epilepsy_data.xml',
    doc='.//document',
    text='.//passage/text/text()',
    id='.//id/text()')

In [3]:
from epilepsy_utils import Tagger
from snorkel.parser import CorpusParser
import os
working_path = os.environ['WORKINGPATH']
dg_tagger = Tagger(working_path + "/Database/epilepsy_tags.tsv")
corpus_parser = CorpusParser(fn=dg_tagger.tag)
%time corpus_parser.apply(list(xml_parser))

Clearing existing...
Running UDF...
[========================================] 99%

TypeError: DeclarativeMeta object argument after ** must be a mapping, not NoneType

In [ ]:
from snorkel.models import Document, Sentence

print "Documents: ", session.query(Document).count()
print "Sentences: ", session.query(Sentence).count()

# Get each candidate relation

This block of code below is designed to gather and tag each sentence found. **Note**: This does include the title of each abstract.

In [ ]:
#This is a quick divide the documents without checking if they have gold standard or not
from snorkel.models import Document
ids = [doc.name for doc in session.query(Document)]
train_ids,dev_ids,test_ids = ids[0:4333],ids[4333:8666],ids[8666:12999]

#Grab the sentences!!!
train_sents,dev_sents,test_sents = set(),set(),set()
docs = session.query(Document).all()
for doc in docs:
    for s in doc.sentences:
        if doc.name in train_ids:
            train_sents.add(s)
        elif doc.name in dev_ids:
            dev_sents.add(s)
        elif doc.name in test_ids:
            test_sents.add(s)
        else:
            print "FAIL"

In [ ]:
from snorkel.models import candidate_subclass

#This specifies that I want candidates that have a disease and gene mentioned in a given sentence
DiseaseGene = candidate_subclass('DiseaseGene', ['Disease', 'Gene'])

In [ ]:
from snorkel.candidates import PretaggedCandidateExtractor

ce = PretaggedCandidateExtractor(DiseaseGene, ['Disease', 'Gene'])

In [ ]:
#Get the candidates from my custom tagger and then print number of candidates found
for k,sents in enumerate([train_sents,dev_sents,test_sents]):
    ce.apply(sents,split=k)
    print "Number of Candidates: ", session.query(DiseaseGene).filter(DiseaseGene.split == k).count()

# Look at the Potential Candidates

The one cool thing about jupyter is that you can use this tool to look at candidates. Check it out after everything above has finished running

In [ ]:
from snorkel.viewer import SentenceNgramViewer

candidates = session.query(DiseaseGene).filter(DiseaseGene.split==0)
sv = SentenceNgramViewer(candidates, session)

In [ ]:
sv